<h1 style="font-size: 48px;">DIGIT CLASSIFICATION (MNIST DATASET)</h1>

<p style="font-size: 20px; color: #666666;">The dataset which is to be needed is available on <a href="https://www.kaggle.com/competitions/digit-recognizer/data">Kaggle</a>.</p>

<p style="font-size:25px; color:black; font-weight:600;">Importing the required libraries.</p>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.python.client import device_lib

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of devices available:", len(physical_devices))
print(device_lib.list_local_devices())
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num of devices available: 1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9517865049169313085
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1734816564
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15137418450109458062
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


<p style="font-size:25px; color:black; font-weight:600;">Importing the dataset from local hard drive.</p>

In [6]:
df_train = pd.read_csv("./train.csv")

In [7]:
df_train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_test = pd.read_csv("./test.csv")

In [9]:
df_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p></p><p style="font-size: 20px; color: #444444;">The dataset obtained is in .csv format and contains images that are converted from image to numerical format in the form of numbers ranging from 0 to 255 inclusive that depicts the brightness of each pixel. The standard image is 28 pixels times 28 pixels format so on conversion there are 784 pixels. The colums depict the brightness of each column.</p><p></p>

In [10]:
X = df_train.iloc[:, 1:].values
y = df_train.iloc[:, 0].values

<p style="font-size:25px; color:black; font-weight:600;">Normalising the dataset.</p>

In [11]:
scaler= StandardScaler()

In [12]:
scaler.fit(X)
X = scaler.transform(X)

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
X_test = df_test

<p style="font-size:25px; color:black; font-weight:600;">Neural Network using Dense layer from Keras.</p>

In [14]:
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [15]:
nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
%%time
nn.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
1050/1050 [==============================] - 3s 2ms/step - loss: 0.3657 - accuracy: 0.8896 - val_loss: 0.1883 - val_accuracy: 0.9448
Epoch 2/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1755 - accuracy: 0.9459 - val_loss: 0.1828 - val_accuracy: 0.9508
Epoch 3/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1382 - accuracy: 0.9581 - val_loss: 0.1683 - val_accuracy: 0.9552
Epoch 4/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1217 - accuracy: 0.9618 - val_loss: 0.1774 - val_accuracy: 0.9558
Epoch 5/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1027 - accuracy: 0.9665 - val_loss: 0.1579 - val_accuracy: 0.9629
Epoch 6/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.0868 - accuracy: 0.9727 - val_loss: 0.2140 - val_accuracy: 0.9612
Epoch 7/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.0830 - accuracy: 0.9730 - val_loss: 0.1534 - val_ac

In [17]:
nny = nn.predict(X_test.values)

875/875 [==============================] - 1s 1ms/step


<p></p><p style="font-size: 20px; color: #444444;">Since the predictions are one-hot encoded, here I am using argmax function of numpy library to return the desired label.</p><p></p>

In [18]:
nny = np.argmax(nny, axis=1)

In [19]:
nny

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [42]:
unique, frequency = np.unique(nny, 
                              return_counts = True)

In [43]:
print("Unique Values:", 
      unique)
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [2710 2600 3032 2998 2261 2188 2823 2771 3695 2922]


<p style="font-size:25px; color:black; font-weight:600;">Random Forest Classifier ensemble from scikit-learn.</p>

<p></p><p style="font-size: 20px; color: #444444;">The main advantage of random forest classifier is its ability to handle high-dimensional input data, such as the pixel intensities in an image. The algorithm can automatically select the most informative features and build decision trees based on them, which can then be combined to make accurate predictions.</p><p></p>

In [27]:
rf = RandomForestClassifier(n_estimators=500, max_depth=1000, n_jobs=8, bootstrap=True, oob_score=True)

In [31]:
%%time
rf.fit(X_train, y_train)

CPU times: total: 1min 9s
Wall time: 28.6 s


RandomForestClassifier(max_depth=1000, n_estimators=500, n_jobs=8,
                       oob_score=True)

In [32]:
rf.score(X_valid, y_valid)

0.965952380952381

In [33]:
%%time
rfy = rf.predict(X_test.values)
rfy.shape

CPU times: total: 672 ms
Wall time: 941 ms


(28000,)

In [45]:
rfy

array([2, 0, 8, ..., 8, 8, 8], dtype=int64)

In [40]:
unique, frequency = np.unique(rfy, 
                              return_counts = True)

In [41]:
print("Unique Values:", 
      unique)
print("Frequency Values:",
      frequency)

Unique Values: [0 2 3 6 8]
Frequency Values: [ 2744   280   108    27 24841]


<p style="font-size:25px; color:black; font-weight:600;">Conclusion</p>

<p></p>
<p style="font-size: 20px; color: #444444;">In this MNIST digit recognizer project, we compared the performance of two machine learning models: a dense neural network and a random forest classifier. Both models were trained on the same dataset of handwritten digit images and evaluated on a held-out test set.</p>
<p style="font-size: 20px; color: #444444;">The results showed that the dense neural network achieved a classification accuracy of 99.52% and validation accuracy of 97.00%, while the random forest classifier achieved a slightly lower accuracy of 96.59%. This suggests that the dense neural network may be slightly more effective at recognizing handwritten digits than the random forest classifier.</p>
<p style="font-size: 20px; color: #444444;">However, it's worth noting that both models achieved very high accuracy scores, indicating that they are both strong candidates for the task of digit recognition. Additionally, the performance of the models could be further improved through various techniques such as hyperparameter tuning, feature engineering, and ensemble learning.</p>
<p style="font-size: 20px; color: #444444;">Overall, this project demonstrates the effectiveness of both dense neural networks and random forest classifiers for the task of MNIST digit recognition, and highlights the importance of evaluating and comparing multiple machine learning models to find the best approach for a given task.</p><p></p>